<a href="https://colab.research.google.com/github/yixish/NLPLearning/blob/master/LSTM_ATT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!/opt/bin/nvidia-smi

Thu Oct 29 14:22:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
import collections
import os
import random
import time
from tqdm import tqdm
import torch
from torch import nn
import torchtext.vocab as Vocab
import torch.utils.data as Data
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
def read_hotel():
    train_df =  pd.read_csv('/content/gdrive/My Drive/dataset/Hotel_rating/train.csv');
    
    keys =  list(train_df['review'].values)
    vals =  list(train_df['rating'].values)
    data = []
    for i in range(len(keys)):
        data.append([keys[i],vals[i]-1])
    data_len = len(data)
    return data[:int(data_len*0.8)],data[int(data_len*0.2):]

train_data,test_data= read_hotel()

# 打印训练数据中的前五个sample
for sample in train_data[:5]:
    print(sample[1], '\t', sample[0][:50])

3 	 good place stay check rainforest biobay vieques/cu
4 	 great firstly did n't enjoy hong kong, 3 days quit
3 	 clean convenient hotel catedral ideally located ke
2 	 transport good class high excellent communications
3 	 stay happy la quinta, used stay travelodge street 


In [ ]:
def get_tokenized(data):
    '''
    @params:
        data: 数据的列表，列表中的每个元素为 [文本字符串，0/1标签] 二元组
    @return: 切分词后的文本的列表，列表中的每个元素为切分后的词序列
    '''
    def tokenizer(text):
        return [tok.lower() for tok in text.split(' ')]
    
    return [tokenizer(review) for review, _ in data]

def get_vocab(data):
    '''
    @params:
        data: 同上
    @return: 数据集上的词典，Vocab 的实例（freqs, stoi, itos）
    '''
    tokenized_data = get_tokenized(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=5)

vocab = get_vocab(train_data)
print('# words in vocab:', len(vocab))

# words in vocab: 14914


In [ ]:
max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

def preprocess(data, vocab):
    '''
    @params:
        data: 同上，原始的读入数据
        vocab: 训练集上生成的词典
    @return:
        features: 单词下标序列，形状为 (n, max_l) 的整数张量
        labels: 情感标签，形状为 (n,) 的0/1整数张量
    '''
 
    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))
 
    tokenized_data = get_tokenized(data)
    features = torch.tensor([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [ ]:
train_set = Data.TensorDataset(*preprocess(train_data, vocab))
test_set = Data.TensorDataset(*preprocess(test_data, vocab))


batch_size = 64
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
print('#batches:', len(train_iter))

torch.int64
torch.int64
X torch.Size([64, 500]) y torch.Size([64])
#batches: 205


In [ ]:
class LSTMATT(nn.Module):

    def __init__(self,vocab, embedding_dim, output_dim, hidden_size, num_layers, bidirectional, dropout):
        super(LSTMATT, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embedding_dim)
        # self.embedding = nn.Embedding.from_pretrained(
        #     pretrained_embeddings, freeze=False)
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout)

        self.fc = nn.Linear(hidden_size * 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.W_w = nn.Parameter(torch.Tensor(hidden_size * 2, hidden_size * 2))
        self.u_w = nn.Parameter(torch.Tensor(hidden_size * 2, 1))

        nn.init.uniform_(self.W_w, -0.1, 0.1)
        nn.init.uniform_(self.u_w, -0.1, 0.1)


    def forward(self, x):
        text = x.T
        # text: [seq_len, batch_size]
        # text_lengths : [batch_size]
        embedded = self.dropout(self.embedding(text))
        # embedded: [seq_len, batch size, emb_dim]

        outputs, _ = self.rnn(embedded)
        # hidden； [num_layers * bidirectional, batch_size, hidden_size]
        # outputs: [real_seq_len, batch_size, hidden_size * 2]

        outputs = outputs.permute(1, 0, 2)
        # outputs: [batch_size, real_seq, hidden_size * 2]

        """ tanh attention 的实现 """
        score = torch.tanh(torch.matmul(outputs, self.W_w))
        # score: [batch_size, real_seq, hidden_size * 2]

        attention_weights = F.softmax(torch.matmul(score, self.u_w), dim=1)
        # attention_weights: [batch_size, real_seq, 1]

        scored_x = outputs * attention_weights
        # scored_x : [batch_size, real_seq, hidden_size * 2]

        feat = torch.sum(scored_x, dim=1)
        # feat : [batch_size, hidden_size * 2]

        return self.fc(feat)

In [ ]:
embed_size, num_hiddens, num_layers = 100, 100, 2
out_dim = 5
model = LSTMATT(vocab,embed_size, out_dim,num_hiddens, num_layers,True,0.5)

In [ ]:
cache_dir = "/content/gdrive/My Drive/dataset/GloVe6B"
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=cache_dir)

def load_pretrained_embedding(words, pretrained_vocab):
    '''
    @params:
        words: 需要加载词向量的词语列表，以 itos (index to string) 的词典形式给出
        pretrained_vocab: 预训练词向量
    @return:
        embed: 加载到的词向量
    '''
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

model.embedding.weight.data.copy_(load_pretrained_embedding(vocab.itos, glove_vocab))
model.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它

There are 5139 oov words.


In [ ]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

def train(train_iter, test_iter, net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [ ]:
lr, num_epochs = 0.01, 20
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()

train(train_iter, test_iter, model, loss, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.1655, train acc 0.486, test acc 0.577, time 39.2 sec
epoch 2, loss 0.4868, train acc 0.556, test acc 0.613, time 39.1 sec
epoch 3, loss 0.3087, train acc 0.588, test acc 0.607, time 38.8 sec
epoch 4, loss 0.2245, train acc 0.602, test acc 0.626, time 38.8 sec
epoch 5, loss 0.1767, train acc 0.606, test acc 0.628, time 38.7 sec
epoch 6, loss 0.1452, train acc 0.612, test acc 0.656, time 38.7 sec
epoch 7, loss 0.1211, train acc 0.620, test acc 0.657, time 38.7 sec
epoch 8, loss 0.1056, train acc 0.624, test acc 0.655, time 38.6 sec
epoch 9, loss 0.0934, train acc 0.624, test acc 0.669, time 38.6 sec
epoch 10, loss 0.0840, train acc 0.630, test acc 0.669, time 38.7 sec
epoch 11, loss 0.0749, train acc 0.633, test acc 0.679, time 38.7 sec
epoch 12, loss 0.0691, train acc 0.631, test acc 0.672, time 38.7 sec
epoch 13, loss 0.0634, train acc 0.635, test acc 0.671, time 38.6 sec
epoch 14, loss 0.0592, train acc 0.630, test acc 0.671, time 38.6 sec
epoch 15, l

In [ ]:
def predict_sentiment(net, vocab, sentence):
    '''
    @params：
        net: 训练好的模型
        vocab: 在该数据集上创建的词典，用于将给定的单词序转换为单词下标的序列，从而输入模型
        sentence: 需要分析情感的文本，以单词序列的形式给出
    @return: 预测的结果，positive 为正面情绪文本，negative 为负面情绪文本
    '''
    if len(sentence)>=max_l:
        sentence = sentence[:max_l]
    device = list(net.parameters())[0].device # 读取模型所在的环境
    sentence = torch.tensor([vocab.stoi[word] for word in sentence], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    return label.item()+1


In [ ]:
test_df =  pd.read_csv('/content/gdrive/My Drive/dataset/Hotel_rating/test.csv');
reviews =  list(test_df['review'].values)
test_df['rating'] = test_df['review'].map(lambda x: predict_sentiment(model,vocab,x.split()))
test_df.describe()

,id,rating
count,4099.000000,4099.000000
mean,2049.000000,4.013174
std,1183.423705,1.211150
min,0.000000,1.000000
25%,1024.500000,4.000000
50%,2049.000000,4.000000
75%,3073.500000,5.000000
max,4098.000000,5.000000


In [ ]:
test_df = test_df.drop('review',axis =1 )
test_df.to_csv('/content/gdrive/My Drive/dataset/sub.csv', header=None,index=False)  